In [1]:
from helper_functions import *

In [2]:
inp = tf.keras.Input(shape=(5))
dense1 = layers.Dense(20, activation=tf.nn.relu, use_bias = True)(inp)
dense2 = layers.Dense(20, activation=tf.nn.relu, use_bias = True)(dense1)
dense3 = layers.Dense(20, activation=tf.nn.relu, use_bias = True)(dense2)
dense4 = layers.Dense(20, activation=tf.nn.relu, use_bias = True)(dense3)
dense5 = layers.Dense(20, activation=tf.nn.relu, use_bias = True)(dense4)
output = layers.Dense(1, use_bias = True)(dense5)
nn_ctlr=tf.keras.Model(inputs=inp,outputs=output)

In [3]:
onnx_model_path = '../benchmarks/controller_5_20.onnx'
onnx_model = onnx.load(onnx_model_path)

weights_list=[]
i=0
for initializer in onnx_model.graph.initializer:
    # Convert the initializer tensor to a NumPy array
    tensor_array = onnx.numpy_helper.to_array(initializer)
    if(i%2==1 and i >=0):
        weights_list.append(tensor_array)
    i = i+1
    
bias_list=[]
i=0
for initializer in onnx_model.graph.initializer:
    # Convert the initializer tensor to a NumPy array
    tensor_array = onnx.numpy_helper.to_array(initializer)
    if(i%2==0 and i >0):
        bias_list.append(tensor_array)
    i = i+1
    
for i in range(1, len(weights_list) + 1):
    transposed_weights = np.transpose(weights_list[i-1])
    combined_weights = [transposed_weights, bias_list[i-1]]
    nn_ctlr.layers[i].set_weights(combined_weights)

In [4]:
weights, biases = extract_model_params_tf(nn_ctlr)

In [5]:
# Change property specifications below
layer_no = 6
neuron_no = 1
time_steps = 5
# output_range = [-1.5,0.55]
input_bounds = [[30,30],[1.4,1.4],[30,30.2],[32,32.2],[10,11],[90,110]]

In [6]:
def summon_gurobi(dec, eqn, log, to, focus=0):
    all_enc = dec + eqn
    file_path = "Gurobi_encodings_5L_ACC.txt"
    with open(file_path, "w") as file:
        for value in all_enc:
            file.write(str(value) + "\n")
    model=gp.Model("Encodings")

    model.Params.MIPFocus = focus
    model.Params.LogToConsole = log
    model.setParam('TimeLimit', to*60*60)
    model.Params.SolutionLimit = 1

    try:
        f = open(file_path,"r")
        try:
            for l in f:
                exec(l)
        finally:
            f.close()
    except IOError:
        pass

    model.optimize()
    
    return model

In [7]:
def can_go_above(w,b,layer,no,time_steps, cond, input_bounds):
    global equations, declare
    equations=[]
    declare=[]
    inputs=w[0].shape[1]

    
    declare.append(f"A0_1_1 = model.addVar(lb={input_bounds[0][0]}, ub={input_bounds[0][1]}, name='A0_1_1')")
    declare.append(f"A0_2_1 = model.addVar(lb={input_bounds[1][0]}, ub={input_bounds[1][1]}, name='A0_2_1')")
    declare.append(f"c = model.addVar(lb={input_bounds[2][0]}, ub={input_bounds[2][1]}, name='c')")
    declare.append(f"d = model.addVar(lb={input_bounds[3][0]}, ub={input_bounds[3][1]}, name='d')")
    declare.append(f"e = model.addVar(lb={input_bounds[4][0]}, ub={input_bounds[4][1]}, name='e')")

    declare.append(f"A0_3_1 = model.addVar(lb={input_bounds[5][0]}, ub={input_bounds[5][1]}, name='A0_3_1')")
    declare.append(f"A0_4_1 = model.addVar(lb=-2.2, ub=-1.8, name='A0_4_1')")
    declare.append(f"A0_5_1 = model.addVar(lb=-100, ub=-79, name='A0_5_1')")

    equations.append(f"model.addConstr(A0_4_1==c-d)")
    equations.append(f"model.addConstr(A0_5_1==e - A0_3_1)")

    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            declare.append(f"P{i}_{j}_0 = model.addVar(name='P{i}_{j}_0')")

    declare.append(f"P{layer}_{no}_0 = model.addVar(name='P{layer}_{no}_0')")

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                declare.append(f"X{i}_{j}_{time} = model.addVar(name='X{i}_{j}_{time}')")
                declare.append(f"P{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY, name='P{i}_{j}_{time}')")
                declare.append(f"S{i}_{j}_{time} = model.addVar(lb=-9999, name='S{i}_{j}_{time}')")
                declare.append(f"q{i}_{j}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{i}_{j}_{time}')")
                declare.append(f"A{i}_{j}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{i}_{j}_{time}')")


    for time in range(1,time_steps+1):
        declare.append(f"S{layer}_{no}_{time} = model.addVar(lb=-9999, name='S{layer}_{no}_{time}')")
    
    # Encodings
    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            equations.append(f"model.addConstr(P{i}_{j}_0== 0)")
    equations.append(f"model.addConstr(P{layer}_{no}_0== 0)")

    thresh = 1
    lamb = 1
    M = 99999
    eps=0.000001

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} + {M}* q{i}_{j}_{time} >= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(X{i}_{j}_{time} >= 0)")
                equations.append(f"model.addConstr(X{i}_{j}_{time} <= {M}*(1-q{i}_{j}_{time}))")
                equations.append(f"model.addConstr(A{i}_{j}_{time} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(A{i}_{j}_{time} + 1 >= X{i}_{j}_{time} + {eps})")
                equations.append(f"model.addConstr(P{i}_{j}_{time} == P{i}_{j}_{time-1} + S{i}_{j}_{time} - A{i}_{j}_{time})")
                equation = f'S{i}_{j}_{time} == ('
                if(i==1):
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_1)'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
                else:
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_{time})'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
    
    out=f''
    for time in range(1,time_steps+1):
        if(time!=1):
            out+= '+'
        out+= f'S{layer}_{no}_{time}'
        
        
        equation = f'S{layer}_{no}_{time} == ('
        for k in range(len(w[layer-1][0])):
            if(k!=0):
                equation += f' + '
            equation+=f'(({w[layer-1][no-1][k]}) * A{layer-1}_{k+1}_{time})'
        equations.append(f"model.addConstr({equation})+ {b[layer-1][no-1]})")
    

    equations.append(f"model.addConstr({out}>={cond[1]*time_steps})")
    equations.append(f'model.setObjective({out}, gp.GRB.MAXIMIZE)')


    return equations, declare 

In [8]:
def can_go_below(w,b,layer,no,time_steps, cond, input_bounds):
    global equations, declare
    equations=[]
    declare=[]
    inputs=w[0].shape[1]

    
    declare.append(f"A0_1_1 = model.addVar(lb={input_bounds[0][0]}, ub={input_bounds[0][1]}, name='A0_1_1')")
    declare.append(f"A0_2_1 = model.addVar(lb={input_bounds[1][0]}, ub={input_bounds[1][1]}, name='A0_2_1')")
    declare.append(f"c = model.addVar(lb={input_bounds[2][0]}, ub={input_bounds[2][1]}, name='c')")
    declare.append(f"d = model.addVar(lb={input_bounds[3][0]}, ub={input_bounds[3][1]}, name='d')")
    declare.append(f"e = model.addVar(lb={input_bounds[4][0]}, ub={input_bounds[4][1]}, name='e')")

    declare.append(f"A0_3_1 = model.addVar(lb={input_bounds[5][0]}, ub={input_bounds[5][1]}, name='A0_3_1')")
    declare.append(f"A0_4_1 = model.addVar(lb=-2.2, ub=-1.8, name='A0_4_1')")
    declare.append(f"A0_5_1 = model.addVar(lb=-100, ub=-79, name='A0_5_1')")

    equations.append(f"model.addConstr(A0_4_1==c-d)")
    equations.append(f"model.addConstr(A0_5_1==e - A0_3_1)")

    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            declare.append(f"P{i}_{j}_0 = model.addVar(name='P{i}_{j}_0')")

    declare.append(f"P{layer}_{no}_0 = model.addVar(name='P{layer}_{no}_0')")

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                declare.append(f"X{i}_{j}_{time} = model.addVar(name='X{i}_{j}_{time}')")
                declare.append(f"P{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY, name='P{i}_{j}_{time}')")
                declare.append(f"S{i}_{j}_{time} = model.addVar(lb=-9999, name='S{i}_{j}_{time}')")
                declare.append(f"q{i}_{j}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{i}_{j}_{time}')")
                declare.append(f"A{i}_{j}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{i}_{j}_{time}')")


    for time in range(1,time_steps+1):
        declare.append(f"S{layer}_{no}_{time} = model.addVar(lb=-9999, name='S{layer}_{no}_{time}')")
    
    # Encodings
    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            equations.append(f"model.addConstr(P{i}_{j}_0== 0)")
    equations.append(f"model.addConstr(P{layer}_{no}_0== 0)")

    thresh = 1
    lamb = 1
    M = 99999
    eps=0.0000001

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} + {M}* q{i}_{j}_{time} >= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(X{i}_{j}_{time} >= 0)")
                equations.append(f"model.addConstr(X{i}_{j}_{time} <= {M}*(1-q{i}_{j}_{time}))")
                equations.append(f"model.addConstr(A{i}_{j}_{time} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(A{i}_{j}_{time} + 1 >= X{i}_{j}_{time} + {eps})")
                equations.append(f"model.addConstr(P{i}_{j}_{time} == P{i}_{j}_{time-1} + S{i}_{j}_{time} - A{i}_{j}_{time})")
                equation = f'S{i}_{j}_{time} == ('
                if(i==1):
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_1)'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
                else:
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_{time})'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
    
    out=f''
    for time in range(1,time_steps+1):
        if(time!=1):
            out+= '+'
        out+= f'S{layer}_{no}_{time}'
        
        
        equation = f'S{layer}_{no}_{time} == ('
        for k in range(len(w[layer-1][0])):
            if(k!=0):
                equation += f' + '
            equation+=f'(({w[layer-1][no-1][k]}) * A{layer-1}_{k+1}_{time})'
        equations.append(f"model.addConstr({equation})+ {b[layer-1][no-1]})")
    

    equations.append(f"model.addConstr({out}<={cond[0]*time_steps})")
    equations.append(f'model.setObjective({out}, gp.GRB.MINIMIZE)')


    return equations, declare

In [9]:
output_range = [-8.819534, -6.850147]

In [12]:
output_range = [-8.819534 , -6.850147]
print(output_range)
tym=5

print('LB Check')
equations, declare = can_go_below(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
model3 = summon_gurobi(declare, equations,0,3,0)
print('\nTime steps',tym)
if(model3.status in [2,10]):
    print('Property does not hold')
elif(model3.status in [9]):
    print('Time out')
else:
    print(model3.status, 'Property holds')
print('Runtime',model3.Runtime)

print('UB Check')
equations, declare = can_go_above(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
model3 = summon_gurobi(declare, equations,0,3,0)
print('\nTime steps',tym)
if(model3.status in [2,10]):
    print('Property does not hold')
elif(model3.status in [9]):
    print('Time out')
else:
    print(model3.status, 'Property holds')
print('Runtime',model3.Runtime)

[-8.819534, -6.850147]
LB Check

Time steps 5
Property does not hold
Runtime 201.4210000038147
UB Check

Time steps 5
Property does not hold
Runtime 74.11400008201599


In [10]:
delta = 0.1
output_range = [-8.819534 - delta , -6.850147 + delta]
print(output_range)
tym=5

print('LB Check')
equations, declare = can_go_below(weights,biases,layer_no,neuron_no,5,output_range, input_bounds)
model3 = summon_gurobi(declare, equations,0,3,0)
print('\nTime steps',tym)
if(model3.status in [2,10]):
    print('Property does not hold')
elif(model3.status in [9]):
    print('Time out')
else:
    print(model3.status, 'Property holds')
print('Runtime',model3.Runtime)

print('UB Check')
equations, declare = can_go_above(weights,biases,layer_no,neuron_no,5,output_range, input_bounds)
model3 = summon_gurobi(declare, equations,0,3,0)
print('\nTime steps',tym)
if(model3.status in [2,10]):
    print('Property does not hold')
elif(model3.status in [9]):
    print('Time out')
else:
    print(model3.status, 'Property holds')
print('Runtime',model3.Runtime)

[-8.819534, -6.850147]
LB Check
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-20

Time steps 5
Property does not hold
Runtime 211.06200003623962
UB Check

Time steps 5
Property does not hold
Runtime 63.66799998283386


In [10]:
delta = 0.2
output_range = [-8.819534 - delta , -6.850147 + delta]
print(output_range)
tym=5
for rotate in range(3):
    print('LB Check')
    equations, declare = can_go_below(weights,biases,layer_no,neuron_no,5,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,0,3,0)
    print('\nTime steps',tym)
    if(model3.status in [2,10]):
        print('Property does not hold')
    elif(model3.status in [9]):
        print('Time out')
    else:
        print(model3.status, 'Property holds')
    print('Runtime',model3.Runtime)

    print('UB Check')
    equations, declare = can_go_above(weights,biases,layer_no,neuron_no,5,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,0,3,0)
    print('\nTime steps',tym)
    if(model3.status in [2,10]):
        print('Property does not hold')
    elif(model3.status in [9]):
        print('Time out')
    else:
        print(model3.status, 'Property holds')
    print('Runtime',model3.Runtime)

[-9.019534, -6.650147]
LB Check
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-20

Time steps 5
3 Property holds
Runtime 46.60699987411499
UB Check

Time steps 5
4 Property holds
Runtime 47.36400008201599
LB Check

Time steps 5
3 Property holds
Runtime 53.955000162124634
UB Check

Time steps 5
4 Property holds
Runtime 47.13100004196167
LB Check

Time steps 5
3 Property holds
Runtime 54.25099992752075
UB Check

Time steps 5
4 Property holds
Runtime 48.19600009918213
